In [7]:
import common_23 as common # This file works in Python 3. video works in python2
import video_23 as video # This file works in Python 3. video works for python2
import os
import numpy as np
import cv2
import random
import time 
import sys
from contextlib import contextmanager
import itertools as it
from os.path import isfile, join

In [8]:
# Convert file folder to video if you have frame sequences instead of a video

def fr2vid(inputpath, outputfile):
    pathIn= inputpath
    pathOut = outputfile
    fps = 32
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    files.sort()
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
    
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

    
#fr2vid('./house/','house.avi') #convert sequence to video

In [9]:
# Homework 3 -- Motion Segmentation helper code

def motion_seg(inputfile):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # In each iteration, calculate absolute difference between current frame and reference frame
        difference = cv2.absdiff(gray, first_gray)

        # Apply thresholding to eliminate noise
        thresh = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=5)
        
        cv2.imshow("thresh", thresh)
        key = cv2.waitKey(5) & 0xFF 
    
        # if the `q` key is pressed, break from the loop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(time.time()-start)

In [10]:
motion_seg('videos/rock.avi')

2.7204179763793945


## Problem II - Motion Segmentation

In [5]:
# the video sequences on which we test the below algorithms
video_sequences = ['rock.avi','beach.avi','jug.avi','lights.avi','railway.avi','trees.avi']

In [6]:
# Non-adaptive frame differencing algorithm

def non_adaptive_frame_differencing(inputfile, threshold):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()
    backgroundFrame = first_gray
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        # In each iteration, calculate absolute difference between current frame and reference frame
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        # Apply thresholding to eliminate noise
        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations = 5)
        
        cv2.imshow("Non-adaptive frame differencing on {0} with threshold = {1}".format(inputfile, threshold), foreground)
        backgroundFrame = currentFrame
        key = cv2.waitKey(5) & 0xFF 
    
        # if the `q` key is pressed, break from the loop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Time taken for threshold {0}: '.format(threshold) + str(time.time() - start))

In [7]:
thresholds = [0, 5, 10, 15, 20, 25, 30]

for seq in video_sequences:
    print('\nFor the {} video sequence:'.format(seq))
    for threshold in thresholds:
        non_adaptive_frame_differencing('videos/' + seq, threshold)


For the rock.avi video sequence:
Time taken for threshold 0: 2.6640098094940186
Time taken for threshold 5: 2.5661139488220215
Time taken for threshold 10: 2.778369903564453
Time taken for threshold 15: 3.0497889518737793
Time taken for threshold 20: 3.1344399452209473
Time taken for threshold 25: 2.9832229614257812
Time taken for threshold 30: 3.0762503147125244

For the beach.avi video sequence:
Time taken for threshold 0: 1.6241412162780762
Time taken for threshold 5: 1.7264900207519531
Time taken for threshold 10: 1.8678529262542725
Time taken for threshold 15: 1.4176647663116455
Time taken for threshold 20: 1.7039477825164795
Time taken for threshold 25: 1.7879207134246826
Time taken for threshold 30: 1.7081005573272705

For the jug.avi video sequence:
Time taken for threshold 0: 1.7637195587158203
Time taken for threshold 5: 1.7560093402862549
Time taken for threshold 10: 1.461838960647583
Time taken for threshold 15: 1.776118516921997
Time taken for threshold 20: 1.471482992172

### Report for Non-adaptive frame differencing

Frame differencing techniques try to improve the naive approach of background subtraction by replacing the background model with the previous image frame.

Non-adaptive frame differencing simply treats the previous image frame as the new background and hence it does not leave behind ghosts of the objects that start and also does not detect the objects that halt. It also adapts to changes in lightning and motion of the camera. But, frame differencing only detects the leading and trailing edges of a uniformly colored moving object which makes it is very hard to detect an object moving towards/away from the camera.

#### Effect of threshold:
By running the above algorithm on different sequences for the following thresholds [0, 5, 10, 15, 20, 25, 30], we observe that:

As the threshold increases, the number of objects which have motion in the output decreases. This is because if we increase the threshold value only objects with significant motion get detected. Low threshold (say 0) treats even small movements as significant while a high threshold (say 30) only filters some objects. We could say that the threshold value of 10 gives good results on average.

In [8]:
# Adaptive frame differencing algorithm
def adaptive_frame_differencing(inputfile, alpha, threshold):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    start = time.time()
    backgroundFrame = first_gray
    i = 1
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        # In each iteration, calculate absolute difference between current frame and reference frame
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        # Apply thresholding to eliminate noise
        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations=5)
        
        cv2.imshow("Adaptive frame differencing on {0} with alpha = {1} and  threshold = {2}".format(inputfile, alpha, threshold), foreground)
        
        backgroundFrame = cv2.addWeighted(currentFrame, alpha, backgroundFrame, 1.0 - alpha, 0)
        key = cv2.waitKey(5) & 0xFF 
    
        # if the `q` key is pressed, break from the loop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print('\tTime taken for threshold {0}: '.format(threshold) + str(time.time() - start))

In [9]:
alpha_values = [1, 0.75, 0.5, 0.25, 0]
thresholds = [10, 20]

for seq in video_sequences:
    print('\nFor the {} video sequence:'.format(seq))
    for alpha in alpha_values:
        print('For alpha = {}'.format(alpha))
        for threshold in thresholds:
            adaptive_frame_differencing('videos/'+ seq, alpha,threshold)


For the rock.avi video sequence:
For alpha = 1
	Time taken for threshold 10: 3.102043867111206
	Time taken for threshold 20: 2.7040255069732666
For alpha = 0.75
	Time taken for threshold 10: 2.841611385345459
	Time taken for threshold 20: 2.34305477142334
For alpha = 0.5
	Time taken for threshold 10: 2.45999813079834
	Time taken for threshold 20: 2.3201980590820312
For alpha = 0.25
	Time taken for threshold 10: 2.805173397064209
	Time taken for threshold 20: 2.3097221851348877
For alpha = 0
	Time taken for threshold 10: 2.534735679626465
	Time taken for threshold 20: 2.3295140266418457

For the beach.avi video sequence:
For alpha = 1
	Time taken for threshold 10: 1.308960199356079
	Time taken for threshold 20: 1.7035503387451172
For alpha = 0.75
	Time taken for threshold 10: 1.7836167812347412
	Time taken for threshold 20: 1.635488748550415
For alpha = 0.5
	Time taken for threshold 10: 1.268000602722168
	Time taken for threshold 20: 1.5056769847869873
For alpha = 0.25
	Time taken for 

### Report for Adaptive frame differencing

Frame differencing techniques try to improve the naive approach of background subtraction by replacing the background model with the previous image frame.


Adaptive frame differencing blends the current image frame into background model using a paramter alpha. If alpha = 0, we get simple bakground subtraction and if alpha = 1 we get non-adaptive frame differencing. As this technique uses alpha as the blending parameter, it gradually eliminates the ghosts of the objects that start and also gradually gets rid of the objects that halt. 

By running the above algorithm on different sequences for the following alpha values [1, 0.75, 0.5, 0.25, 0] and the following thresholds [10, 20, 30], we observe that:

#### Effect of alpha:
As alpha decreases, the time taken for ghosts to blend into the background increases. Also, alpha = 1 produces the same result as non-adaptive frame differencing algorithm with the same threshold. Further, aplha = 0 leaves behind ghosts as it is equivalent to simple background subtraction. Overall, the alpha value of 0.5 seems to produce good results.

#### Effect of threshold:
As the threshold increases, the number of objects which have motion in the output decreases. This is because if we increase the threshold value only objects with significant motion get detected.

In [18]:
# Persistent frame differencing algorithm

def persistent_frame_differencing(inputfile, gamma, threshold):
    cap = video.create_capture(inputfile)
    ret, first = cap.read()

    # For background subtraction, Save the first image as reference
    first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
    first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)
    
    backgroundFrame = first_gray
    
    h, w = first_gray.shape[:2]
    motion_history = np.zeros((h, w),np.float32)
    start = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
    
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        currentFrame = gray
        
        # In each iteration, calculate absolute difference between current frame and reference frame
        foreground = cv2.absdiff(backgroundFrame, currentFrame)

        # Apply thresholding to eliminate noise
        foreground = cv2.threshold(foreground, threshold, 255, cv2.THRESH_BINARY)[1]
        foreground = cv2.dilate(foreground, None, iterations = 5)
        
        motion_history = motion_history - gamma
        motion_history[motion_history < 0] = 0
        tmp = motion_history
        
        foreground = 255 * foreground
        
        for i in range(h):
            for j in range(w):
                if foreground[i, j] > tmp[i, j]:
                    motion_history[i,j] = foreground[i, j]
                else:
                    motion_history[i,j] = tmp[i, j] 
        
        cv2.imshow("Persistent frame differencing on {0} with gamma = {1} and threshold = {2}".format(inputfile, gamma, threshold) ,motion_history)
        backgroundFrame = currentFrame
        
        key = cv2.waitKey(1) & 0xFF 
    
        # if the `q` key is pressed, break from the loop
        if key == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print('Time taken for threshold {0}: '.format(threshold) + str(time.time() - start))

In [22]:
persistent_frame_differencing('videos/rock.avi', 0.25, 10)

Time taken for threshold 10: 11.896270036697388


#### Uncomment to run for all sequences and different values of gamma and threshold

In [23]:
# gamma_values = [0, 0.125, 0.25, 0.375, 0.5]
# thresholds = [10, 20]

# for seq in video_sequences:
#     print('\nFor the {} video sequence:'.format(seq))
#     for threshold in thresholds:
#         persistent_frame_differencing('videos/' + seq, threshold)

### Report for Persistant frame differencing

In this technique we use motion history of the image and mark a pixel with the last time it was detected foreground. This gives us a sense of motion with some kind of fading trail of the objects that move.

Since the background image is updated using the current image frame the ghosts fade away. It also adapts to changes in motion of the camera and illumination.

#### Effect of Gamma:
By running the algorithm on the following gamma values [0, 0.125, 0.25, 0.375, 0.5] we see that as the value of gamma decreases, the motion history of the image is clearly seen i.e the objects have longer trails. But, gamma value of 0 will retain the trail of the object as a ghost. As gamma increases the we no longer see the motion history. Of the given values gamma value of 0.25 seems optimal. 

#### Effect of threshold:
By running the above algorithm on different sequences for the following thresholds [10, 20], we observe that:

As the threshold increases, the number of objects which have motion in the output decreases. This is because if we increase the threshold value only objects with significant motion get detected. Low threshold (say 10) treats even small movements as significant while a high threshold (say 20) only filters bodies with large motion. We could say that the threshold value of 10 gives good results on average.